In [29]:
import pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GMM 
from featureextraction import extract_features
#from speakerfeatures import extract_features
import warnings
warnings.filterwarnings("ignore")
import os
import time



In [30]:
source      = "/Users/jm186072/Documents/DataScience/Speaker-Identification-Python/trng_data/"  
test_data_folder   = "/Users/jm186072/Documents/DataScience/Speaker-Identification-Python/test_data/"  
dest        = "/Users/jm186072/Documents/DataScience/Speaker-Identification-Python/models/"

#train_file  = "/Users/jm186072/Documents/DataScience/Speaker-Identification-Python/trngdata.txt"


In [31]:
#file_paths = open(train_file,'r')


In [32]:
features = np.asarray(())

for dirs in os.listdir(source):
    if dirs == '.DS_Store':
        continue
    #print(dirs)
    dirpath = source+dirs
    #print(dirpath)
    count=0
    for audiofiles in os.listdir(dirpath+'/wav'):

        if audiofiles == '.DS_Store':
            continue
        
        count = count + 1
        #print(count,audiofiles)
        
        
        sr,audio = read(dirpath+'/wav'+'/'+audiofiles)

        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)
        
        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
        #print(audiofiles)
        #print(features.shape)
        if count ==5:
            
            #print("Creating GMM ")
            gmm = GMM(n_components = 16, n_iter = 200, covariance_type='diag',n_init = 3)
            gmm.fit(features)
            #print(type(features))
            #print(features.shape)
            # dumping the trained gaussian model
            
            picklefile = str(dest.strip()+dirs.strip()+ ".gmm")
            #print(picklefile)
            cPickle.dump(gmm,open(picklefile,'wb'))
            
            print ('speaker:->',dirs,"Data points = ",features.shape)

            count=0
            features = np.asarray(())
            break
            
        
        
    

speaker:-> calamity-20071011-poe Data points =  (2711, 40)
speaker:-> chocoholic-20070524 Data points =  (2443, 40)
speaker:-> chocoholic-20070612-eti33 Data points =  (2775, 40)
speaker:-> chocoholic-20080420-pos Data points =  (3499, 40)
speaker:-> cloudmountain-20080420-eti Data points =  (1910, 40)
speaker:-> cloudmountain-20080420-yos Data points =  (4613, 40)
speaker:-> crhylove-10252006 Data points =  (2437, 40)
speaker:-> delibab-20071007-poe Data points =  (2809, 40)
speaker:-> delibab-20071012 Data points =  (3150, 40)
speaker:-> delibab-20071019-poe Data points =  (3202, 40)
speaker:-> delibab-20071020 Data points =  (3442, 40)
speaker:-> delibab-20071025-poe Data points =  (3915, 40)
speaker:-> ductapeguy-20070308b Data points =  (1323, 40)
speaker:-> ductapeguy-20070619-com Data points =  (3007, 40)
speaker:-> ductapeguy-20080423-pri Data points =  (3227, 40)
speaker:-> ductapeguy-20080423-sto Data points =  (3906, 40)
speaker:-> gesine-20080421-rid Data points =  (4184, 4

In [33]:
modelpath = "/Users/jm186072/Documents/DataScience/Speaker-Identification-Python/models/"

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]
#print(gmm_files,len(gmm_files))

In [37]:
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]

spk_list   = [fname.split("/")[-1].split(".gmm")[0] for fname in gmm_files]

#print(spk_list)

In [38]:
# Code for Identifying an audio 
test_dir = r'/Users/jm186072/Documents/DataScience/Speaker-Identification-Python/test1/'
total_sample = 0
for test_audio_file in os.listdir(test_dir):
    
    if test_audio_file == '.DS_Store':
        continue
    
    total_sample += 1.0
    
    sr,audio = read(test_dir+test_audio_file)
    vector = extract_features(audio,sr)
    log_likelihood = np.zeros(len(models))
    
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()

    winner = np.argmax(log_likelihood)
    
    print (test_audio_file,"is detected as - ", spk_list[winner])  


as0010.wav is detected as -  calamity-20071011-poe
as0011.wav is detected as -  calamity-20071011-poe
as0012.wav is detected as -  calamity-20071011-poe
as0013.wav is detected as -  calamity-20071011-poe


In [36]:
total_sample = 0
error = 0

for speakers in os.listdir(test_data_folder):
    if speakers == '.DS_Store':
        continue
    #print(speakers)
    for audio_file in os.listdir(test_data_folder+speakers+'/wav'):
        if audio_file == '.DS_Store':
            continue
        #print('\t',audio_file)
        total_sample += 1 
        sr,audio = read(test_data_folder+speakers+'/wav/'+audio_file)
        vector = extract_features(audio,sr)
        
        log_likelihood = np.zeros(len(models))
        
                    
        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()

        winner = np.argmax(log_likelihood)
        #print ("\tdetected as - ", speakers[winner])
        #print(winner)
        if spk_list[winner] != speakers:
            error += 1
            
print ('Total Audio Files Processed = ',total_sample,'Incorrectly Identified = ' , error )
print('Identification Accuracy -->',((total_sample - error) / total_sample) * 100)

Total Audio Files Processed =  1362 Incorrectly Identified =  192
Identification Accuracy --> 85.90308370044053
